In [3]:
import numpy as np
import pandas as pd
from datetime import date
import collections
import datetime
import os
import xarray as xr
import csv

## Generic the filepath to the main data folder
fpath0 = '/Users/finnwimberly/Library/CloudStorage/GoogleDrive-fwimberly@middlebury.edu/My Drive/'
fpath1 = 'Lizz Research Stuff/CSV Outputs/Initial Areas/Results_PET_PREC/'

modelnames = ['BCC.BCC-CSM2-MR', 'MPI-M.MPI-ESM1-2-HR', 'MRI.MRI-ESM2-0', 'NCAR.CESM2-WACCM']

scenarios = ['Ssp2p6', 'Ssp4p5', 'Ssp7p0', 'Ssp8p5']

basins = {'RHINE':'6242', 'RHONE':'6243','PO':'6241', 'DANUBE':'6202', 'TITICACA':'3912', 'SANTA':'3425', 
            'OCONA':'3418', 'MAJES':'3416', 'MAGDALENA':'3227', 'AMAZON':'3203', 'YELCHO':'3429', 
            'VALDIVIA':'3428', 'SERRANO':'3426', 'RAPEL':'3423', 'PUELO':'3422', 'PASCUA':'3420', 
            'PALENA':'3419', 'HUASCO':'3412', 'COPIAPO':'3409', 'CISNES':'3408', 'BIOBIO':'3405', 'BAKER':'3404',
            'AZOPARDO':'3403', 'AISEN':'3401', 'SANTA CRUZ':'3244', 'NEGRO':'3232', 'COLORADO':'3212', 
            'CHICO':'3209', 'TORNEALVEN':'6255', 'THJORSA':'6254', 'OLFUSA':'6237', 'LULEALVEN':'6227', 
            'KUBAN':'6223', 'KALIXALVEN':'6219', 'GLOMAA':'6213', 'DRAMSELVA':'6209', 'SVARTA':'6110', 
            'LAGARFLJOT':'6104', 'JOKULSA A FJOLLUM':'6101', 'CLUTHA':'5406', 'YUKON':'4435', 'TAKU':'4431', 
             'SUSITNA':'4430','STIKINE':'4428', 'SKEENA':'4427','SKAGIT':'4426','NUSHAGAK':'4418','NASS':'4416',
            'KUSKOKWIM':'4414','FRASER':'4410', 'COPPER':'4408', 'COLUMBIA':'4406', 'ALSEK':'4401', 'NELSON':'4125', 
              'MACKENZIE':'4123','COLVILLE':'4110', 'YSYK-KOL':'2919', 'UVS NUUR':'2918', 'TARIM HE':'2914', 
              'TALAS':'2913', 'LAKE BALKHASH':'2910','HAR US NUUR':'2909', 'CHUY':'2905', 'ARAL SEA':'2902', 
              'YELLOW RIVER':'2434', 'MEKONG':'2421', 'KAMCHATKA':'2413', 'SALWEEN':'2319', 'IRRAWADDY':'2310', 
              'INDUS':'2309', 'GANGES':'2306','BRAHMAPUTRA':'2302', 'OB':'2108', 'INDIGIRKA':'2103','YANGTZE' : '2433'}

In [4]:
#Loading in files
precip = {}
for m, model in enumerate(modelnames):
    precip[model[0:3]] = {}
    for s, SSP in enumerate(scenarios):
        precip[model[0:3]][SSP] = np.loadtxt(fpath0 + fpath1 + model + '_PREC_' + SSP + '.txt')

PET = {}
for m, model in enumerate(modelnames):
    PET[model[0:3]] = {}
    for s, SSP in enumerate(scenarios):
        PET[model[0:3]][SSP] = np.loadtxt(fpath0 + fpath1 + model + '_PET_' + SSP + '.txt')

In [5]:
# Create new index using pandas date_range function
start_date = datetime.date(1900, 1, 1)
end_date = datetime.date(2100, 12, 1)
timesteps = pd.date_range(start_date, end_date, freq='MS').strftime('%Y-%m').tolist()

# Apply new index and datetime conversion
for m, model in enumerate(modelnames):
    for s, SSP in enumerate(scenarios):
        precip[model[0:3]][SSP] = pd.DataFrame(precip[model[0:3]][SSP])
        precip[model[0:3]][SSP].index = basins
        precip[model[0:3]][SSP].columns = timesteps

        PET[model[0:3]][SSP] = pd.DataFrame(PET[model[0:3]][SSP])
        PET[model[0:3]][SSP].index = basins
        PET[model[0:3]][SSP].columns = timesteps

In [6]:
#Examining how many PETs are negative
negative_PETs = {}
percent_PET_negative = {}
for m, model in enumerate(modelnames):
    negative_PETs[model[0:3]] = {}
    percent_PET_negative[model[0:3]] = {}
    for s, SSP in enumerate(scenarios):
        negative_PETs[model[0:3]][SSP] = PET[model[0:3]][SSP][PET[model[0:3]][SSP]<0]
        negative_PETs[model[0:3]][SSP] = negative_PETs[model[0:3]][SSP].stack().reset_index()
        #negative_PETs[model[0:3]][SSP] = negative_PETs[model[0:3]][SSP].stack().reset_index()
        negative_PETs[model[0:3]][SSP].columns = ['Basins', 'Date', 'PET']
        percent_PET_negative[model[0:3]][SSP] = (len(negative_PETs[model[0:3]][SSP])/(2412 * 75)) * 100

In [7]:
for m, model in enumerate(modelnames):
    for s, SSP in enumerate(scenarios):
        print(model, SSP, percent_PET_negative[model[0:3]][SSP])

BCC.BCC-CSM2-MR Ssp2p6 20.137092316196796
BCC.BCC-CSM2-MR Ssp4p5 19.974018794914315
BCC.BCC-CSM2-MR Ssp7p0 19.84466556108347
BCC.BCC-CSM2-MR Ssp8p5 19.755113322277502
MPI-M.MPI-ESM1-2-HR Ssp2p6 15.088999447208403
MPI-M.MPI-ESM1-2-HR Ssp4p5 15.02377003869541
MPI-M.MPI-ESM1-2-HR Ssp7p0 14.906578220011054
MPI-M.MPI-ESM1-2-HR Ssp8p5 14.878938640132668
MRI.MRI-ESM2-0 Ssp2p6 16.671641791044774
MRI.MRI-ESM2-0 Ssp4p5 16.47705914870094
MRI.MRI-ESM2-0 Ssp7p0 16.34825870646766
MRI.MRI-ESM2-0 Ssp8p5 16.206744057490326
NCAR.CESM2-WACCM Ssp2p6 15.55721393034826
NCAR.CESM2-WACCM Ssp4p5 15.440022111663904
NCAR.CESM2-WACCM Ssp7p0 15.384190160309563
NCAR.CESM2-WACCM Ssp8p5 15.163626312880044


In [8]:
# Setting up time ranges
start_1900 = pd.to_datetime("1900-01-01")
end_2000 = pd.to_datetime("1999-12-31")
start_2000 = pd.to_datetime("2000-01-01")
end_2100 = pd.to_datetime("2100-12-31")

# Initialize dictionaries to store summed values
PET_avg_1900_2000 = {}
PET_avg_2000_2100 = {}
precip_avg_1900_2000 = {}
precip_avg_2000_2100 = {}

# Loop through models and scenarios
for m, model in enumerate(modelnames):
    PET_avg_1900_2000[model[0:3]] = {}
    PET_avg_2000_2100[model[0:3]] = {}
    precip_avg_1900_2000[model[0:3]] = {}
    precip_avg_2000_2100[model[0:3]] = {}
    for s, SSP in enumerate(scenarios):
        # Filter columns for the period 1900-2000
        columns_1900_2000 = [col for col in PET[model[0:3]][SSP].columns if start_1900 <= pd.to_datetime(col) <= end_2000]
        # Sum values for the period 1900-2000
        PET_avg_1900_2000[model[0:3]][SSP] = pd.DataFrame({"Avg_1900_1999":PET[model[0:3]][SSP][columns_1900_2000].sum(axis = 1)})/1200

        columns_2000_2100 = [col for col in PET[model[0:3]][SSP].columns if start_2000 <= pd.to_datetime(col) <= end_2100]
        PET_avg_2000_2100[model[0:3]][SSP] = pd.DataFrame({"Avg_2000_2100": PET[model[0:3]][SSP][columns_2000_2100].sum(axis = 1)})/1212

        columns_1900_2000 = [col for col in precip[model[0:3]][SSP].columns if start_1900 <= pd.to_datetime(col) <= end_2000]
        # Avg values for the period 2000-2100
        precip_avg_1900_2000[model[0:3]][SSP] = pd.DataFrame({"Avg_1900_1999":precip[model[0:3]][SSP][columns_1900_2000].sum(axis = 1)})/1200

        columns_2000_2100 = [col for col in PET[model[0:3]][SSP].columns if start_2000 <= pd.to_datetime(col) <= end_2100]
        precip_avg_2000_2100[model[0:3]][SSP] = pd.DataFrame({"Avg_2000_2100":precip[model[0:3]][SSP][columns_2000_2100].sum(axis = 1)})/1212

In [9]:
aridity_index_20thCent = {}
aridity_index_21stCent = {}
for m, model in enumerate(modelnames):
    aridity_index_20thCent[model[0:3]] = {}
    aridity_index_21stCent[model[0:3]] = {}
    for s, SSP in enumerate(scenarios):
         aridity_index_20thCent[model[0:3]][SSP] = precip_avg_1900_2000[model[0:3]][SSP] / PET_avg_1900_2000[model[0:3]][SSP]
         aridity_index_21stCent[model[0:3]][SSP] = precip_avg_2000_2100[model[0:3]][SSP] / PET_avg_2000_2100[model[0:3]][SSP]

In [10]:
#If I want to scale can use this
#For now values look ok
# from sklearn.preprocessing import MinMaxScaler

# aridity_index = {}
# for m, model in enumerate(modelnames):
#     aridity_index[model[0:3]] = {}
#     for s, SSP in enumerate(scenarios):

#         temp_df =  (precip[model[0:3]][SSP] / PET[model[0:3]][SSP])
#         scaler = MinMaxScaler(feature_range=(0, 100))
#         aridity_index[model[0:3]][SSP] = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=basins)


In [11]:
# Define the directory to save the CSV files
output_dir = '/Users/finnwimberly/Desktop/Lizz Research/CSV Outputs/Basin Aridity/'

models = ['BCC', 'MPI', 'MRI', 'NCA']

for m, GCM in enumerate(models):
    for s, SSP in enumerate(scenarios):
        fname = f"Aridity_Index_Monthly{GCM}_{SSP}.csv"

        # Define the full path of the output file
        output_path = os.path.join(output_dir, fname)

        # Save the DataFrame as CSV
        aridity_index_21stCent[GCM][SSP].to_csv(output_path, header=True, index=True)

In [48]:
#Making df which contains all values to calulate an average
combined_df = pd.DataFrame()

GCM_SSP_indices = []
for m, GCM in enumerate(models):
    for s, SSP in enumerate(scenarios):
        column_name = f'{GCM}_{SSP}'
        GCM_SSP_indices.append(column_name)

# Loop through models and scenarios
for model_key in aridity_index_21stCent.keys():
    for s, SSP in enumerate(scenarios):
        # Extract the dataframe for the current model and scenario
        current_df = aridity_index_21stCent[model_key][SSP]
        
        # Add the current_df as a new column in the combined_df
        if combined_df.empty:
            combined_df = current_df
        else:
            combined_df = pd.concat([combined_df, current_df], axis=1)

# Set the column names of the combined_df to be the values in GCM_SSP_indices
combined_df.columns = GCM_SSP_indices

# Calculate the average of the aridity index across the columns
average_aridity_index = combined_df.mean(axis=1)

# Add the average as a new column in the combined_df
combined_df['Average_Aridity_Index'] = average_aridity_index

In [52]:
fname = f"Aridity_Index_Avg.csv"

# Define the full path of the output file
output_path = os.path.join(output_dir, fname)

# Save the DataFrame as CSV
average_aridity_index.to_csv(output_path, header=True, index=True)